<a href="https://colab.research.google.com/github/vksepm/build-ai-agents-and-chatbots-with-langgraph-2021112/blob/main/code_02_XX%20Basic%20ReAct%20Agent%20in%20LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install all dependency packages for the course
#Remember to execute this before running any of the exercises
!pip install -qU tenacity==9.0.0
!pip install -qU langchain==0.3.12
!pip install -qU langchain-openai==0.2.12
!pip install -qU langchain_community==0.3.12
!pip install -qU langgraph==0.2.59
!pip install -qU pysqlite3-binary==0.5.4
!pip install -qU langchain_chroma==0.1.4
!pip install -qU pandas==2.2.3
!pip install -qU pypdf==5.1.0
!pip install -qU nbformat==5.10.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━

## 2.3. Setup Function Tools for ReAct Agent

In [2]:
from langchain_core.tools import tool

#Tool annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y


## 2.4. Create a basic ReAct Agent

In [3]:
from langchain_openai import AzureChatOpenAI
import os
from google.colab import userdata
#Setup the LLM for the agent



os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["OPENAI_API_VERSION"] = userdata.get('OPENAI_API_VERSION')

#Setup the LLM
model = AzureChatOpenAI(model="gpt-4o-mini")

#Test the model
#response = llm.invoke("Hello, how are you?")
#print(response.content)


In [4]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available.
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model=model,
    state_modifier=system_prompt,
    tools=agent_tools)


## 2.5. Execute the ReAct Agent

In [5]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The sum of 2 and 3 is 5. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (call_wC106SiWuGEEG8axT7tShFqG)
 Call ID: call_wC106SiWuGEEG8axT7tShFqG
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


In [6]:
#Example 2
inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 1 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The result of 3 multiplied by 2 is 6, and 5 plus 1 is also 6. 

Step by Step execution : 
================================ Human Message =================================

What is 3 multipled by 2 and 5 + 1 ?
================================== Ai Message ==================================
Tool Calls:
  find_product (call_dXTZMRBQUkTKrzVYfFIEdMn9)
 Call ID: call_dXTZMRBQUkTKrzVYfFIEdMn9
  Args:
    x: 3
    y: 2
  find_sum (call_CWGBQLGLN83DS5ip8peLu1aW)
 Call ID: call_CWGBQLGLN83DS5ip8peLu1aW
  Args:
    x: 5
    y: 1
================================= Tool Message =================================
Name: find_product

6
================================= Tool Message =================================
Name: find_sum

6
================================== Ai Message ==================================

The result of 3 multiplied by 2 is 6, and 5 plus 1 is also 6.


## 2.6. Debugging the Agent

In [7]:
agent_graph=create_react_agent(
    model=model,
    state_modifier=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'what is the sum of 2 and 3 ?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'what is the sum of 2 and 3 ?')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='0edf7b7f-890c-467b-89d3-75f73029250e')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='0edf7b7f-890c-467b-89d3-75f73029250e')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tke1hcpGiLY2koDNOxJVjQMV', 'function': {'arguments': '{"x":2,"y":3}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}